# 1. 문자 문제

 - 인코딩  
 코드 포인트 ==> 바이트  
 
 
   
    
 - 디코딩  
 바이트 ==> 코드 포인트

In [6]:
s = 'cafⓔ'
len(s)

4

In [7]:
b = s.encode('utf-8')
b

b'caf\xe2\x93\x94'

In [8]:
len(b)

6

In [10]:
b.decode('utf-8')

'cafⓔ'

# 2. 바이트에 대한 기본 지식

위에서 보듯이 bytes는 str에 인코딩을 지정해서 만들 수 있습니다. bytes의 각 항목은 range(256)에 속하는 정수이며, bytes는 슬라이싱해도 bytes인데, 이는 슬라이스가 한 바이트일 때도 마찬가지입니다.

bytesarray에 대한 리터럴 구문은 없고, bytes 리터럴을 인수로 사용해서 bytearray()로 표현합니다.

In [11]:
cafe = bytes('cafⓔ', encoding = 'utf-8')
cafe

b'caf\xe2\x93\x94'

In [12]:
cafe[0]

99

In [13]:
cafe[:1]

b'c'

In [15]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xe2\x93\x94')

In [16]:
cafe_arr[-1:]

bytearray(b'\x94')

바이너리 시퀀스가 실제로 정수 타입의 시퀀스이기는 하지만, 리터럴 표기법을 보면 실제로는 아스키 텍스트가 들어가는 경우가 많다는 것을 알 수 있습니다. 따라서 각 바이트 값에 따라 다음과 같이 세 가지 형태로 출력이 가능합니다.

 - 화면에 출력 가능한 아스키 문자
 - 탭, 개행 문자, 캐리지 리턴(carriage return), 백슬래시(\)는 이스케이프 시퀀스(\t, \n, \r, \\)로 출력
 - 그 외의 값을 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력

In [24]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

바이너리 시퀀스는 fromhex()라는 str에 없는 클래스 메소드도 제공하는데, 이 메소드를 이용하면 공백으로 구분된 16진수 쌍을 파싱해서 바이너리 시퀀스를 만들 수 있습니다.

In [26]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

여기서 'h' 타입코드는 short int(16비트) 타입의 배열을 생성합니다. octets은 numbers를 구성하는 바이트들의 사본을 가지고 있으며 octets은 다섯 개의 short int 타입을 나타내는 10바이트입니다.

### 2.1 구조체와 메모리 뷰

struct 모듈은 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석하고, 이와 반대로 튜플을 패킹된 바이트로 변환하는 함수를 제공한다. struct는 bytes, bytearray, memoryview 객체와 함께 사용된다.

In [36]:
# memoryview와 struct를 사용해서 GIF 이미지 헤더 조사하기
import struct
fmt = '<3s3sHH'
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())
header = img[:10]
bytes(header)

b'<!DOCTYPE '

In [39]:
struct.unpack(fmt, header) # 종류 버전 너비 높이 튜플로 언패킹 

(b'<!D', b'OCT', 20569, 8261)

In [40]:
del header
del img

# 3. 기본 인코더/디코더

텍스트를 바이트 혹은 바이트를 텍스트로 변환하기 위해 파이썬에는 100여개의 코덱(인코더/디코더)이 포함되어 있다. 각 코덱은 utf_8과 같은 이름을 갖고 있는데, utf_8은 utf8, utf-8, U8 등으로 불리기도 한다. 코덱은 open(), str.encode(), bytes.decode() 등의 함수를 호출할 때, encoding 인수에 전달해서 사용할 수 있다.

In [42]:
# 전혀 다른 바이트 시퀀스를 만드는 세 개의 코덱으로 인코딩한 El Niño 문자열
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep = '\t')
    

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


# 인코딩/디코딩 문제 이해하기

UnicodeError 라는 범용 예외가 있긴 하지만, 일반적으로 UnicodeEncodeError, UnicodeDecodeError 같은 구체적인 예외가 발생한다. 유니코드 에러가 발생할 때, 이러한 정확한 유형을 파악하는 것이 중요하다.

### 4.1 UnicodeEncodeError 처리하기

UnicodeEncodeError 는 비UTF 코덱이 유니코드 문자의 일부만을 처리할 수 있으므로, 문자가 인코딩에 정의되어 있지 않아 발생하는 에러이다.


In [53]:
# 바이트로 인코딩하기 : 성공 및 에러 처리
city = ' São Paulo'
city.encode('utf_8')

b' S\xc3\xa3o Paulo'

In [45]:
city.encode('utf_16')

b'\xff\xfe \x00S\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [46]:
city.encode('iso8859_1')

b' S\xe3o Paulo'

In [49]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 2: character maps to <undefined>

In [50]:
city.encode('cp437', errors='ignore')

b' So Paulo'

In [51]:
city.encode('cp437', errors='replace')

b' S?o Paulo'

In [52]:
city.encode('cp437', errors='xmlcharrefreplace')

b' S&#227;o Paulo'

### 4.2 UnicodeDecodeError 처리하기

UnicodeEncodeError 는 비UTF 코덱이 유니코드 문자의 일부만을 처리할 수 있으므로, 문자가 인코딩에 정의되어 있지 않아 발생하는 에러이다.

UnicodeDecodeError 는 이진 시퀀스를 텍스트로 변환할 때, 문자로 변환할 수 없으면 발생한다. 하지만 'cp1252', 'iso8859_1' 등 많은 레거시 8비트 코덱은 무작위 비트 배열에 대해서도 에러를 발생시키지 않고 디코딩할 수 있다. 이 때 왜곡된 문자를 그렘린(gremlin) 혹은 문자 깨짐이라고 한다.



In [57]:
#str에서 bytes로 디코딩하기: 성공 및 에러 처리
octets = b'Montr\xe9al'
octets.decode('cp1252')

'Montréal'

In [58]:
octets.decode('iso8859_7')

'Montrιal'

In [59]:
octets.decode('koi8_r')

'MontrИal'

In [62]:
octets.decode('utf_8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [63]:
octets.decode('utf_8', errors='replace')

'Montr�al'

### 4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError

파이썬 3버전 부터는 UTF-8을 기본 인코딩 방식으로 사용한다. 인코딩 선언 없이 비UTF-8로 인코딩된 모듈을 로딩하면 SyntaxError가 발생한다. 

In [65]:
# Olã.py '헬로 월드!' 포르투갈어 버전
#coding: cp1252

print('Olã Mundo!')

Olã Mundo!


### 4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법

바이트 시퀀스의 인코딩 방식을 알아낼 수는 없다. 별도의 인코딩 정보를 가져와야 한다. Chardet 이라는 프로그램은 인코딩 방식에 따른 결과의 특징을 통해서 역으로 인코딩 방식을 추정한다.

UTF 인코딩을 통한 바이트 시퀀스는 BOM(byte order mark) 이라는 바이트 순서 표시를 추가함으로써, 인코딩 방식이 UTF 방식이라는 것을 유추할 수 있다. 하지만 파이썬은 BOM이 추가되어있다고 하더라도, 자동으로 UTF-8로 인코딩되어 있다고 가정하지는 않는다.

 

### 4.5 BOM: 유용한 깨진 문자

In [66]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

UTF-16으로 인코딩된 텍스트 앞에 여분의 바이트가 있는 것을 보았을 것이다.
b'\xff\xfe' 문자가 앞에 나온 것을 볼 수 있다. 이 문자가 바로 바이트 순서 표시(BOM)로, 인코딩한 인텔 CPU의 '리틀엔디언' 바이트 순서를 나타낸다.

In [67]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

# 5. 텍스트 파일 다루기

텍스트를 처리하는 가장 좋은 방법은 유니코드 샌드위치다. 


![이미지 설명](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbQv0Gp%2FbtrhesueUgx%2Fxmyvy5hktRjNZQVlIAGRF0%2Fimg.png)
이는 입력할 때 가능하면 빨리 bytes를 str로 변환해야 한다는 것을 의미한다. 샌드위치에 들어가는 고기는 프로그램의 비즈니스 논리에 해당하는 부분이며, 여기서는 텍스트를 오로지 str 객체로 다룬다. 출력할 때는 가능한 한 늦게 str을 bytes로 인코딩한다.

파이썬 3의 open() 함수는 파일을 텍스트 모드로 읽고 쓸 때 필요한 모든 인코딩과 디코딩 작업을 수행해주므로 my_file.read() 에서 str 객체를 가져와서 처리하고 my_file.write() 에 전달하면 된다.

 

이 때, 실행하는 컴퓨터에 따라 인코딩 방식을 지정해주지 않으면 버그가 발생할 수 있다. 윈도우의 경우 기본 인코딩이 UTF8이 아닐 수 있기 때문이다.

 

가장 중요한 점은 컴퓨터나 OS 환경에 따르는 기본 인코딩에 의존하지 않는 것이다. 항상 인코딩 방식을 명시하는 것이 좋다.

In [75]:
open('cafe.txt', 'w', encoding='utf_8').write('cafⓔ')

4

In [79]:
# 플랫폼 인코딩 문제
open('cafe.txt').read()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 3: illegal multibyte sequence

In [81]:
#윈도우에서 실행해서 조사하면 버그와 해결 방법을 찾을 수 있다.
fp = open('cafe.txt', 'w', encoding='utf_8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf_8'>

In [82]:
fp.write('cafⓔ')

4

In [83]:
fp.close()

In [84]:
import os
os.stat('cafe.txt').st_size

6

In [85]:
fp2 = open('cafe.txt')
fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp949'>

In [86]:
fp2.encoding

'cp949'

In [87]:
fp2.read()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 3: illegal multibyte sequence

In [88]:
fp3 = open('cafe.txt', encoding='utf_8')
fp3

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf_8'>

In [89]:
fp3.read()

'cafⓔ'

In [90]:
fp4 = open('cafe.txt', 'rb')
fp4

<_io.BufferedReader name='cafe.txt'>

In [91]:
fp4.read()

b'caf\xe2\x93\x94'

### 5.1 기본 인코딩 설정: 정신 나간 거 아냐?

대충 리눅에서는 utf-8인코딩을 사용하지만 윈도우에서는 cp1252를 사용한다는 이야기

# 6 제대로 비교하기 위해 유니코드 정규화 하기

유니코드에는 결합 문자가 있기 때문에 문자열 비교가 쉽지 않다. 발음 구별 기호는 앞 문자와 하나로 결합되어 출력된다.

In [92]:
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(len(s1), len(s2))
print(s1 == s2)

café café
4 5
False


유니코드 표준에서는 'é' 와 '\u0301' 이 두개의 시퀀스를 규범적으로 동일하다고 보며, 애플리케이션은 이 두 시퀀스를 동일하게 처리해야 하지만 파이썬에서는 서로 동일하지 않다고 판단한다.

 

이 때, unicodedata.normalize() 함수가 제공하는 정규화를 이용해야 한다. 이 함수의 첫 번째 인수는 'NFC', 'NFD', 'NFKC', 'NFKD' 중 하나여야 한다.

 

NFC(Normalization Form C) 는 코드 포인트를 조합해서 가장 짧은 동일 문자열을 생성하는 반면, NFD는 조합된 문자를 기본 문자와 별도의 결합 문자로 분리한다.

In [93]:
from unicodedata import normalize
ss1 = 'café'
ss2 = 'cafe\u0301'

print(len(normalize('NFC', ss1)), len(normalize('NFC', ss2)))

print(len(normalize('NFD', ss1)), len(normalize('NFD', ss2)))

print(normalize('NFC', ss1) == normalize('NFC', ss2))

print(normalize('NFD', ss1) == normalize('NFD', ss2))

4 4
5 5
True
True


키보드는 일반적으로 결합된 문자를 입력할 수 있으므로, 사용자가 입력하는 텍스트는 기본적으로 NFC 형태다. 그러나 안전을 보장하기 위해 파일에 저장하기 전 normalize('NFC', user_text) 코드로 문자열을 청소하는 것이 좋다.

그럼에도 NFC에 의해 서로 다르게 정규화 되는 문자가 있다. 전기 저항을 나타내는 문자 Ω (옴) 기호는 그리스어 대문자 오메가로 정규화 된다.

나머지 두 인수 NFKC 와 NFKD 에서의 K는 호환성(compatibility)을 나타낸다. 정규화의 더 강력한 형태로서, 호환성 문자에 영향을 미친다. 위 방식에서 각 호환성 문자는 포매팅 손실이 발생하더라도 선호하는 형태의 하나 이상의 문자로 구성된 호환성 분할로 치환된다. 예를 들면,  절반을 나타내는 '½' 문자의 호환성 분할은 세 개 문자의 시퀀스인 '1/2' 로 치환된다. 즉, 일반적으로 많이 사용되고 사용자들이 선호하는 형태로 분할 및 치환됨을 의미한다.

In [94]:
from unicodedata import normalize, name

half = '½'
print(normalize('NFKC', half))

1⁄2


NFKC와 NFKD 는 검색 및 색인 생성을 위한 편리한 중간 형태를 생성할 수 있는 장점이 있지만, 정보를 왜곡하여 데이터가 손실될 수 있기 때문에 영구 저장할 경우에는 사용하지 않는 것이 좋다.

### 6.1 케이스 폴딩

케이스 폴딩은 모든 텍스트를 소문자로 변환하는 연산이며, 약간의 변환을 동반한다.


In [97]:
micro = 'µ'
print(name(micro))

micro_cf = micro.casefold()
print(name(micro_cf))

MICRO SIGN
GREEK SMALL LETTER MU


### 6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수

In [98]:
#라틴 문자에서 결합 표시 기호를 제거하는 함수
import unicodedata
import string

greek = 'Ζέφυρος, Zéfiro'

def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>

print(shave_marks_latin(greek))

Ζέφυρος, Zefiro


In [99]:
#서양 활자(타이포그래픽) 기호를 아스키로 변환하기)
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>

# 7.유니코드 텍스트 정렬하기

파이썬은 각 시퀀스 안에 들어 있는 항목들을 하나하나 비교하여 어떤 자료형의 시퀀스도 설명할 수 있고, 문자열의 경우에는 각 단어의 코드 포인트를 비교한다. 바이스키 문자를 사용하는 경우 부적절한 결과가 발생할 수 있다.

In [102]:
fruits = ['caju','atemoia','cajȧ', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajȧ']

In [104]:
#locale.strxfrm() 함수를 정렬키로 사용하기
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [105]:
fruits = ['caju','atemoia','cajȧ', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key = locale.strxfrm)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajȧ', 'caju']

### 7.1 유니코드 대조 알고리즘을 이용한 정렬

장고에 다양한 기여를 하고 있는 제임스토버도 유니코드 대조 알고리즘을 순수 파이썬으로 구현한 PyUCA를 만들었다.

In [108]:
import pyuca

coll = pyuca.Collator()
fruits = ['caju','atemoia','cajȧ', 'açaí', 'acerola']
sorted_fruits =  sorted(fruits, key=locale.strxfrm)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajȧ', 'caju']

# 8. 유니코드 데이터베이스

유니코드 표준은 수 많은 구조화된 텍스트 파일의 형태로 하나의 완전한 데이터베이스를 제공한다.
이 데이터베이스엔 코드 포인트를 문자명으로 매핑하는 테이블뿐만 아니라 각 문자에 대한 메타데이터 및 각 문자의 연관 방법을 담고 있다.

str의 isidentifier(), isprintable(), isdecimal(), isnumeric() 메서드는 이 데이터베이스를 사용하고, str.casfold() 메서드도 유니코드 테이블의 정보를 사용한다.

unicodedata 모듈에는 문자 메타데이터를 반환하는 함수들로, 표준에 정의된 공식 몇잉, 결합 문자인지 연부, 사람이 인식하는 기호의 숫자값 등을 반환한다

In [112]:
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),
        char.center(6),
        're_dig' if re_digit.match(char) else '-',
        'isdig' if char.isdigit() else '-',
        'isnum' if char.isnumeric() else '-',
        format(unicodedata.numeric(char), '5.2f'),
        unicodedata.name(char),
        sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


# 9. 이중 모드 str 및 bytes API

표준 라이브러리에는 str이나 bytes 인수를 모두 받을 시, 인수의 자료형에 따라 다르게 작용하는 함수들이 있고, re 와 os 모듈이 대표적이다.



### 9.1 정규 표현식에서 str과 bytes 
정규 표현식에서의 str과 bytes
bytes로 정규 표현식을 만들 경우 \d와 \w같은 패턴은 아스키 문자만 매칭되고, str로 이 패턴을 만들 시 아스키 문자 외외에 유니코드 숫자나 문자도 매칭된다.

In [134]:
#간단한 str과 bytes 정규 표현식의 동작 비교


# 정규 표현식을 str과 bytes에 사용할 수 있지만,
# bytes에 정규 표현식을 사용하면 아스키 범위를 벗어나는  문자들은 숫자나 단어로 처리하지 않는다.
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0be7\u0bed\u0be8\u0bef" 
            "as 1729 = 1^3 + 12^3 = 9^3 + 10^3.")

text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n ')
print('Numbers')
print(' str :', re_numbers_str.findall(text_str))
print(' bytes :', re_numbers_bytes.findall(text_bytes))
print('Words')
print(' str :', re_words_str.findall(text_str))
print(' bytes :', re_words_bytes.findall(text_bytes))

Text
 'Ramanujan saw ௧௧௭௨௯as 1729 = 1^3 + 12^3 = 9^3 + 10^3.'
Numbers
 str : ['௧௧௭௨௯', '1729', '1', '3', '12', '3', '9', '3', '10', '3']
 bytes : [b'1729', b'1', b'3', b'12', b'3', b'9', b'3', b'10', b'3']
Words
 str : ['Ramanujan', 'saw', '௧௧௭௨௯as', '1729', '1', '3', '12', '3', '9', '3', '10', '3']
 bytes : [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'3', b'12', b'3', b'9', b'3', b'10', b'3']


### 9.2 os 모듈 함수에서 str과 bytes
GNU/리눅스 커널은 유니코드를 모르기에 실제 os의 파일명은 어떠한 인코딩 체계에서도 올바르지 않은 바이트 시퀀스로 구성되어 있기에 str로 디코딩 할 수 없고, 다양한 운영 체계를 클라이언트로 가지는 파일 서버는 이런 문제가 발생하기 쉽다.



해결은 파일명이나 경로명을 받는 모든 os 모듈 함수는 str이나 bytes형의 인수를 받고 str 인수로 호출하면 인수는 sys.getfilesystemencoding() 함수에 의해 지정된 코덱을 이용해서 자동으로 변환되고, 운영 체계의 응답은 동일 코덱을 이용해서 디코딩 되므로 유니코드 샌드위치 모델에 따라서 원하는대로 가능

In [139]:
#str과 bytes 인수로 호출한 listdir()메서드와 결과

os.listdir('.')

['.ipynb_checkpoints',
 'cafe.txt',
 'ch2. 시퀀스.ipynb',
 'ch3. 딕셔너리와 집합.ipynb',
 'ch4. 텍스트와 바이트.ipynb',
 'digits-of-π.txt',
 'filter.gif',
 'floats.bin']

In [140]:
os.listdir(b'.')

[b'.ipynb_checkpoints',
 b'cafe.txt',
 b'ch2. \xec\x8b\x9c\xed\x80\x80\xec\x8a\xa4.ipynb',
 b'ch3. \xeb\x94\x95\xec\x85\x94\xeb\x84\x88\xeb\xa6\xac\xec\x99\x80 \xec\xa7\x91\xed\x95\xa9.ipynb',
 b'ch4. \xed\x85\x8d\xec\x8a\xa4\xed\x8a\xb8\xec\x99\x80 \xeb\xb0\x94\xec\x9d\xb4\xed\x8a\xb8.ipynb',
 b'digits-of-\xcf\x80.txt',
 b'filter.gif',
 b'floats.bin']

 - **fsencode(파일명)**  
파일명이 str형이면 sys.getfilesystemencoding()이 변환한 코덱명을 이용해서 파일명을 bytes형으로 인코딩한다. 그러나,파일명이 bytes형이면 변환하지 않고 그대로 반환한다.
-----
 - **fsdecode(파일명)**
 파일명이 bytes 형이면 sys.getfilesystemencoding()이 반환한 코덱명을 이용해서 파일명을 str형으로 디코딩하고, 파일명이 str형이면 변환하지 않고 그대로 반환한다.

##### surrogateescape를 이용해서 깨진 문자 처리하기
예상치 못한 bytes나 모르는 인코딩을 처리하기 위한 바이트 제안서의 설명을 바탕으로 python에선 surrogateescape 코덱 에러 처리기 사용

In [144]:
"""
디코딩할 수 없는 바이트를 유니코드 표준에서 하위 써로게이트 영역이라고 코드 포인트로 치환한다.

애플리케이션 내부 용도로 사용할 수 있도록 한다.
"""

os.listdir('.')

['.ipynb_checkpoints',
 'cafe.txt',
 'ch2. 시퀀스.ipynb',
 'ch3. 딕셔너리와 집합.ipynb',
 'ch4. 텍스트와 바이트.ipynb',
 'digits-of-π.txt',
 'filter.gif',
 'floats.bin']

In [145]:
os.listdir(b'.')

[b'.ipynb_checkpoints',
 b'cafe.txt',
 b'ch2. \xec\x8b\x9c\xed\x80\x80\xec\x8a\xa4.ipynb',
 b'ch3. \xeb\x94\x95\xec\x85\x94\xeb\x84\x88\xeb\xa6\xac\xec\x99\x80 \xec\xa7\x91\xed\x95\xa9.ipynb',
 b'ch4. \xed\x85\x8d\xec\x8a\xa4\xed\x8a\xb8\xec\x99\x80 \xeb\xb0\x94\xec\x9d\xb4\xed\x8a\xb8.ipynb',
 b'digits-of-\xcf\x80.txt',
 b'filter.gif',
 b'floats.bin']

In [153]:
pi_name_bytes = os.listdir(b'.')[5]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str # 치환된 원래 바이트로 다시 변환 

'digits-of-\udccf\udc80.txt'